In [ ]:
from singa import tensor
from singa.tensor import Tensor
from singa import autograd
from singa import optimizer
import numpy as np

In [9]:
autograd.training = True

# prepare training data in numpy array

# generate the boundary
f = lambda x: (5 * x + 1)
bd_x = np.linspace(-1., 1, 200)
bd_y = f(bd_x)
# generate the training data
x = np.random.uniform(-1, 1, 400)
y = f(x) + 2 * np.random.randn(len(x))
# convert training data to 2d space
label = np.asarray([5 * a + 1 > b for (a, b) in zip(x, y)])
data = np.array([[a, b] for (a, b) in zip(x, y)], dtype=np.float32)

def to_categorical(y, num_classes):
    '''
    Converts a class vector (integers) to binary class matrix.
    Args
        y: class vector to be converted into a matrix
            (integers from 0 to num_classes).
        num_classes: total number of classes.
    Return
        A binary matrix representation of the input.
    '''
    y = np.array(y, dtype='int')
    n = y.shape[0]
    categorical = np.zeros((n, num_classes))
    categorical[np.arange(n), y] = 1
    return categorical

label = to_categorical(label, 2).astype(np.float32)
print('train_data_shape:', data.shape)
print('train_label_shape:', label.shape)

inputs = Tensor(data=data)
target = Tensor(data=label)

w0 = Tensor(shape=(2, 2), requires_grad=True, stores_grad=True)
w0.gaussian(0.0, 0.1)
b0 = Tensor(shape=(1, 2), requires_grad=True, stores_grad=True)
b0.set_value(0.0)

w1 = Tensor(shape=(2, 2), requires_grad=True, stores_grad=True)
w1.gaussian(0.0, 0.1)
b1 = Tensor(shape=(1, 2), requires_grad=True, stores_grad=True)
b1.set_value(0.0)

w2 = Tensor(shape=(2, 2), requires_grad=True, stores_grad=True)
w2.gaussian(0.0, 0.1)
b2 = Tensor(shape=(1, 2), requires_grad=True, stores_grad=True)
b2.set_value(0.0)

sgd = optimizer.SGD(0.05)

train_data_shape: (400, 2)
train_label_shape: (400, 2)


In [3]:
#sgd.apply?

In [19]:
# training process
for i in range(1001):
    x = autograd.matmul(inputs, w0)
    x = autograd.add_bias(x, b0)
    x = autograd.relu(x)
    x = autograd.matmul(x, w1)
    x = autograd.add_bias(x, b1)
    x = autograd.soft_max(x)
    loss = autograd.cross_entropy(x, target)
    #print(autograd.backward(loss))
    for p, gp in autograd.backward(loss).items():
        #print(p.shape)
        #print(gp.shape)
        gp.reshape(p.shape)
        #print()
        #gp = gp.reshape(p.shape)
        #print(gp.shape)
        sgd.apply(0, gp, p, '')

    if (i % 100 == 0):
        print('training loss = ', tensor.to_numpy(loss)[0])

training loss =  0.6918451
training loss =  0.6151072
training loss =  0.58906007
training loss =  0.572035
training loss =  0.53840005
training loss =  0.42323416
training loss =  0.31211025
training loss =  0.23464933
training loss =  0.18611358
training loss =  0.1552738
training loss =  0.13001356


In [ ]:
for i in range(1):
    x = autograd.matmul(inputs, w0)
    x = autograd.add_bias(x, b0)
    #x = autograd.relu(x)
    x2 = autograd.matmul(x, w2)
    x2 = autograd.add_bias(x2, b2)
    x1 = autograd.matmul(x, w1)
    x1 = autograd.add_bias(x1, b1)
    x3 = autograd.add(x1,x2)
    x3 = autograd.softmax(x3)
    loss = autograd.cross_entropy(x3, target)
    for p, gp in autograd.backward(loss):
        #gp.reshape(p.shape)
        pass
        #sgd.apply(0, gp, p, '')
    print('training loss = ', tensor.to_numpy(loss)[0])